# Bronze Layer : Raw JSON to delta table

In [0]:
import os
from pyspark.sql.functions import col, split, substring_index, sum
from pyspark.sql.types import StructType, StructField, StringType

# --- 1. Configuration ---
# The root of the recreated cvelistV5 structure
root_path = "/Volumes/workspace/default/assignment1/recreated_cvelistV5"

print(f"Starting file count analysis for: {root_path}")

# --- 2. List All Files Recursively ---
# This uses a fast, schema-based read to get all file paths.
file_list_df = (spark.read
    .format("text")
    .schema(StructType([StructField("path", StringType(), True)]))
    .option("recursiveFileLookup", "true")
    .load(os.path.join(root_path, "cves"))
)

# --- 3. Extract Year and Filter for JSON Files ---
# Filter out non-JSON files (like .DS_Store, etc.) and extract the year from the path.
df_with_year = (file_list_df
    .filter(col("path").like("%.json"))
    .withColumn("year", split(col("path"), "/")[7]) 
)

# --- 4. Count Files Per Year ---
print("\n--- Files Counted Per Year ---")

# Group by the extracted year and count the files.
df_count_by_year = (df_with_year
    .groupBy("year")
    .count()
    .withColumnRenamed("count", "file_count")
    .orderBy(col("year").asc())
)

# Display the results for all years
df_count_by_year.show(df_count_by_year.count(), truncate=False)

# --- 5. Calculate Grand Total ---
# Sum up the 'file_count' column from our aggregated DataFrame.
grand_total = df_count_by_year.agg(sum("file_count")).collect()[0][0]

print("\n--- Overall File Count ---")
print(f"Total JSON files found across all years: {grand_total}")
print("--------------------------")

Starting file count analysis for: /Volumes/workspace/default/assignment1/recreated_cvelistV5

--- Files Counted Per Year ---
+----+----------+
|year|file_count|
+----+----------+
+----+----------+


--- Overall File Count ---
Total JSON files found across all years: None
--------------------------


In [0]:
from pyspark.sql.functions import col, year
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, TimestampType

# --- 1. Use complete and correct schema for maximum data quality ---
cve_schema = StructType([
    StructField("dataType", StringType(), True),
    StructField("dataVersion", StringType(), True),
    StructField("cveMetadata", StructType([
        StructField("cveId", StringType(), True),
        StructField("datePublished", StringType(), True),
        StructField("dateUpdated", StringType(), True)
    ])),
    StructField("containers", StructType([
        StructField("cna", StructType([
            StructField("title", StringType(), True),
            StructField("descriptions", ArrayType(StructType([
                StructField("value", StringType(), True)
            ]))),
            StructField("affected", ArrayType(StructType([
                StructField("vendor", StringType(), True),
                StructField("product", StringType(), True)
            ]))),
            StructField("metrics", ArrayType(StructType([
                StructField("cvssV3_1", StructType([
                    StructField("baseScore", DoubleType(), True),
                    StructField("baseSeverity", StringType(), True)
                ]), True),
                StructField("cvssV4_0", StructType([
                    StructField("baseScore", DoubleType(), True),
                    StructField("baseSeverity", StringType(), True)
                ]), True)
            ])))
        ]))
    ]))
])

# --- 2. Configuration and Setup ---
source_path = "/Volumes/workspace/default/assignment1/recreated_cvelistV5/cves"
output_table_name = "cve_bronze.records"
NUM_OUTPUT_FILES = 16 

spark.sql("CREATE SCHEMA IF NOT EXISTS cve_bronze")
print(f"Reading all JSON files recursively from: {source_path}")

# --- 3. Run the Ingestion and Filtering Job ---
try:
    print("\nStarting parallel ingestion job...")
    
    # The full pipeline: Read -> Filter -> Coalesce
    df_optimized = (spark.read
        .schema(cve_schema)
        .option("recursiveFileLookup", "true")
        .json(source_path)
        .filter(col("cveMetadata.cveId").isNotNull())
        .filter(year(col("cveMetadata.datePublished").cast(TimestampType())) == 2024)
        .coalesce(NUM_OUTPUT_FILES)
    )

    print("Data transformation plan is defined. Proceeding to write.")
    
    # --- 4. Write the DataFrame to the Bronze Delta Table ---
    print(f"\nWriting data to the Delta table: {output_table_name}")
    print("!!! MONITOR THE SPARK UI PROGRESS BAR BELOW THIS CELL !!!")
    
    (df_optimized.write
        .option("overwriteSchema", "true") 
        .mode("overwrite")
        .saveAsTable(output_table_name)
    )

    print("\nBronze layer ingestion complete!")
    
    # --- 5. Data Quality Checks and Verification---
    print(f"\n--- Verifying the final table '{output_table_name}' ---")
    
    final_table = spark.table(output_table_name)
    record_count = final_table.count()
    print(f"SUCCESS: Ingested {record_count} records into {output_table_name}.")
    
    assert record_count >= 30000, f"DATA QUALITY FAILED: Expected >= 30,000 records, found {record_count}."
    print("✅ Quality Check Passed: Record count is above threshold.")
    
    print("\n--- Final Bronze Table Sample ---")
    display(final_table.limit(10))

except Exception as e:
    print(f"AN ERROR OCCURRED: {e}")

Reading all JSON files recursively from: /Volumes/workspace/default/assignment1/recreated_cvelistV5/cves

Starting parallel ingestion job...
Data transformation plan is defined. Proceeding to write.

Writing data to the Delta table: cve_bronze.records
!!! MONITOR THE SPARK UI PROGRESS BAR BELOW THIS CELL !!!

Bronze layer ingestion complete!

--- Verifying the final table 'cve_bronze.records' ---
SUCCESS: Ingested 40274 records into cve_bronze.records.
✅ Quality Check Passed: Record count is above threshold.

--- Final Bronze Table Sample ---


dataType,dataVersion,cveMetadata,containers
CVE_RECORD,5.1,"List(CVE-2014-5470, 2024-06-21T00:00:00, 2024-08-06T11:48:49.046Z)","List(List(null, List(List(Actual Analyzer through 2014-08-29 allows code execution via shell metacharacters because untrusted input is used for part of the input data passed to an eval operation.)), List(List(n/a, n/a)), null))"
CVE_RECORD,5.1,"List(CVE-2015-10123, 2024-03-13T08:31:55.341Z, 2024-10-23T09:40:12.408Z)","List(List(Wago: Buffer Copy without Checking Size of Input in wbm of multiple products, List(List(An unautheticated remote attacker could send specifically crafted packets to a affected device. If an authenticated user then views that data in a specific page of the web-based management a buffer overflow will be triggered to gain full access of the device.)), List(List(WAGO, Controller BACnet/IP), List(WAGO, Controller BACnet MS/TP), List(WAGO, Ethernet Controller 3rd Generation), List(WAGO, Ethernet Controller 3rd Generation), List(WAGO, Fieldbus Coupler Ethernet 3rd Generation)), List(List(List(8.8, HIGH), null))))"
CVE_RECORD,5.1,"List(CVE-2015-10128, 2024-01-02T13:31:03.041Z, 2025-04-17T18:38:12.123Z)","List(List(rt-prettyphoto Plugin rt-prettyphoto.php royal_prettyphoto_plugin_links cross site scripting, List(List(A vulnerability was found in rt-prettyphoto Plugin up to 1.2 on WordPress and classified as problematic. Affected by this issue is the function royal_prettyphoto_plugin_links of the file rt-prettyphoto.php. The manipulation leads to cross site scripting. The attack may be launched remotely. Upgrading to version 1.3 is able to address this issue. The patch is identified as 0d3d38cfa487481b66869e4212df1cefc281ecb7. It is recommended to upgrade the affected component. VDB-249422 is the identifier assigned to this vulnerability.), List(Eine Schwachstelle wurde in rt-prettyphoto Plugin bis 1.2 für WordPress gefunden. Sie wurde als problematisch eingestuft. Dies betrifft die Funktion royal_prettyphoto_plugin_links der Datei rt-prettyphoto.php. Durch Manipulieren mit unbekannten Daten kann eine cross site scripting-Schwachstelle ausgenutzt werden. Der Angriff kann über das Netzwerk passieren. Ein Aktualisieren auf die Version 1.3 vermag dieses Problem zu lösen. Der Patch wird als 0d3d38cfa487481b66869e4212df1cefc281ecb7 bezeichnet. Als bestmögliche Massnahme wird das Einspielen eines Upgrades empfohlen.)), List(List(n/a, rt-prettyphoto Plugin)), List(List(List(3.5, LOW), null), List(null, null), List(null, null))))"
CVE_RECORD,5.1,"List(CVE-2015-10129, 2024-02-04T04:31:03.305Z, 2024-08-06T08:58:26.521Z)","List(List(planet-freo auth.inc.php comparison, List(List(A vulnerability was found in planet-freo up to 20150116 and classified as problematic. Affected by this issue is some unknown functionality of the file admin/inc/auth.inc.php. The manipulation of the argument auth leads to incorrect comparison. The attack may be launched remotely. The complexity of an attack is rather high. The exploitation is known to be difficult. The exploit has been disclosed to the public and may be used. This product is using a rolling release to provide continious delivery. Therefore, no version details for affected nor updated releases are available. The name of the patch is 6ad38c58a45642eb8c7844e2f272ef199f59550d. It is recommended to apply a patch to fix this issue. The identifier of this vulnerability is VDB-252716.), List(Eine Schwachstelle wurde in planet-freo bis 20150116 gefunden. Sie wurde als problematisch eingestuft. Es geht hierbei um eine nicht näher spezifizierte Funktion der Datei admin/inc/auth.inc.php. Mit der Manipulation des Arguments auth mit unbekannten Daten kann eine incorrect comparison-Schwachstelle ausgenutzt werden. Der Angriff kann über das Netzwerk angegangen werden. Die Komplexität eines Angriffs ist eher hoch. Sie ist schwierig ausnutzbar. Der Exploit steht zur öffentlichen Verfügung. Dieses Produkt verzichtet auf eine Versionierung und verwendet stattdessen Rolling R

In [0]:
print(record_count)

40274


In [0]:
%sql
-- This command shows the metadata of the Delta table, proving it was created correctly.
DESCRIBE DETAIL cve_bronze.records;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,164206fb-50c9-48dd-a443-b8a859faa847,workspace.cve_bronze.records,null,,2025-11-11T10:41:16.422Z,2025-11-11T11:53:04.000Z,List(),List(),13,6037554,"Map(delta.parquet.compression.codec -> zstd, delta.enableDeletionVectors -> true)",3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false
